**1 - Configurando o Spark no Python**

Etapa responsável por realizar o Download e configuração do Apache Spark.

In [ ]:
#Instalação e configuração do Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Download do Apache Spark
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#Descompactando o Apache Spark
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#Instalação do Findspark
!pip install -q findspark

**2 - Configurando o Ambiente**

Etapa responsável por configurar o ambiente do Spark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

**3 - Configurando a sessão do Spark**

Etapa responsável por inicializar a sessão do Spark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

**4 - Validando a Configuração e Versão do Spark**

Etapa de Verificação

In [ ]:
import pyspark
print(pyspark.__version__)

3.0.1


**5 - Download dos Dados do Portal da Transparência**

Etapa responsável por baixar os dados do Cadastro Nacional de Empresas Idôneas e Suspensas providas através da API do Portal da Transparência

In [ ]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados: fe3446f262599c3a5918efe5b05afd43' 'http://www.portaltransparencia.gov.br/api-de-dados/ceis?dataInicialSancao=01%2F01%2F2020&dataFinalSancao=22%2F10%2F2020&pagina=1' > consulta_portal_transparencia.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31026    0 31026    0     0  13857      0 --:--:--  0:00:02 --:--:-- 13857


**6 - Carregando os dados no Spark**

Etapa para a carga de dados no Spark.

In [ ]:
#Instancia o contexto do Spark
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Governo Federal sobre o PETI
path = "/content/consulta_portal_transparencia.json"
#Declara dataframe com o conteúdo da API baixada
df = spark.read.json(path)

**7 - Explorando os dados que foram obtidos**

Etapa de exploração dos dados

In [ ]:
#Imprime a estrutura de dados do dataframe
df.printSchema()
#Imprime uma amostra de dados
df.show()

root
 |-- abrangenciaDefinidaDecisaoJudicial: string (nullable = true)
 |-- dataFimSancao: string (nullable = true)
 |-- dataInicioSancao: string (nullable = true)
 |-- dataOrigemInformacao: string (nullable = true)
 |-- dataPublicacaoSancao: string (nullable = true)
 |-- dataReferencia: string (nullable = true)
 |-- dataTransitadoJulgado: string (nullable = true)
 |-- detalhamentoPublicacao: string (nullable = true)
 |-- fonteSancao: struct (nullable = true)
 |    |-- enderecoContato: string (nullable = true)
 |    |-- nomeExibicao: string (nullable = true)
 |    |-- telefoneContato: string (nullable = true)
 |-- id: long (nullable = true)
 |-- informacoesAdicionaisDoOrgaoSancionador: string (nullable = true)
 |-- legislacao: struct (nullable = true)
 |    |-- descricaoFundamentacaoLegal: string (nullable = true)
 |    |-- fundamentacaoLegal: string (nullable = true)
 |-- linkPublicacao: string (nullable = true)
 |-- numeroProcesso: string (nullable = true)
 |-- orgaoSancionador: stru

**8 - Tratando e transformando os dados**

In [ ]:
dfPrepared = df.selectExpr("abrangenciaDefinidaDecisaoJudicial as Abrangencia", "dataFimSancao", "dataInicioSancao","dataTransitadoJulgado", "fonteSancao", "orgaoSancionador","pessoa", "tipoSancao")
dfPrepared.show()

+--------------------+-------------+----------------+---------------------+--------------------+--------------------+--------------------+--------------------+
|         Abrangencia|dataFimSancao|dataInicioSancao|dataTransitadoJulgado|         fonteSancao|    orgaoSancionador|              pessoa|          tipoSancao|
+--------------------+-------------+----------------+---------------------+--------------------+--------------------+--------------------+--------------------+
|Todas as Esferas ...|   31/05/2020|      02/03/2020|           02/03/2020|[Rua Duque de Cax...|[PREFEITURA MUNIC...|[[Comércio atacad...|[Inidoneidade - L...|
|      Sem Informação|   22/10/2020|      25/06/2020|       Sem informação|[Praça Municipal ...|[TRIBUNAL REGIONA...|[[Comércio vareji...|[Impedimento - Le...|
|      Sem Informação|   22/10/2020|      22/04/2020|       Sem informação|[Rua Siqueira Cam...|[Governo do Estad...|[[Serviços de eng...|[Suspensão - Legi...|
|Todas as Esferas ...|   31/05/2020|    

**9 - Análise das informações e resultados obtidos**

Etapa onde são realizadas as análises utilizando linguagem SQL

In [ ]:
#Declara tabela temporária no SparkSQL
dfPrepared.createOrReplaceTempView("dfPrepared")

In [ ]:
#Empresas sem informação de abrangência
emp_seminfo = spark.sql("select * from dfPrepared where abrangencia = 'Sem Informação'")
emp_seminfo.show()

+--------------+-------------+----------------+---------------------+--------------------+--------------------+--------------------+--------------------+
|   Abrangencia|dataFimSancao|dataInicioSancao|dataTransitadoJulgado|         fonteSancao|    orgaoSancionador|              pessoa|          tipoSancao|
+--------------+-------------+----------------+---------------------+--------------------+--------------------+--------------------+--------------------+
|Sem Informação|   22/10/2020|      25/06/2020|       Sem informação|[Praça Municipal ...|[TRIBUNAL REGIONA...|[[Comércio vareji...|[Impedimento - Le...|
|Sem Informação|   22/10/2020|      22/04/2020|       Sem informação|[Rua Siqueira Cam...|[Governo do Estad...|[[Serviços de eng...|[Suspensão - Legi...|
|Sem Informação|   22/10/2020|      22/04/2020|       Sem informação|[Rua Siqueira Cam...|[Governo do Estad...|[[Sem informação,...|[Suspensão - Legi...|
|Sem Informação|   22/10/2020|      22/07/2020|       Sem informação|[Av. Er

In [ ]:
#Empresas com Data de trânsito julgado
emp_data= spark.sql("select * from dfPrepared where dataTransitadoJulgado <> 'Sem informação'")
emp_data.show()

+--------------------+-------------+----------------+---------------------+--------------------+--------------------+--------------------+--------------------+
|         Abrangencia|dataFimSancao|dataInicioSancao|dataTransitadoJulgado|         fonteSancao|    orgaoSancionador|              pessoa|          tipoSancao|
+--------------------+-------------+----------------+---------------------+--------------------+--------------------+--------------------+--------------------+
|Todas as Esferas ...|   31/05/2020|      02/03/2020|           02/03/2020|[Rua Duque de Cax...|[PREFEITURA MUNIC...|[[Comércio atacad...|[Inidoneidade - L...|
|Todas as Esferas ...|   31/05/2020|      02/03/2020|           02/03/2020|[Rua Duque de Cax...|[PREFEITURA MUNIC...|[[Representantes ...|[Inidoneidade - L...|
|No órgão sancionador|   15/10/2020|      15/04/2020|           22/04/2020|[Avenida Engenhei...|[Governo do Estad...|[[Sem informação,...|[Inidoneidade - L...|
|Todas as Esferas ...|   31/05/2020|    

In [ ]:
#Qtde empresas por Órgão Sancionador
emp_pororgao= spark.sql("select count(*), orgaoSancionador from dfPrepared group by 2 order by 1 desc")
emp_pororgao.show()

+--------+--------------------+
|count(1)|    orgaoSancionador|
+--------+--------------------+
|       4|[Governo do Estad...|
|       3|[PREFEITURA MUNIC...|
|       1|[SECRETARIA DA SA...|
|       1|[Governo do Estad...|
|       1|[Secretaria de Es...|
|       1|[TRIBUNAL REGIONA...|
+--------+--------------------+

